In [1]:
from datetime import datetime as dt

import const
import json
import pandas as pd
import numpy

In [2]:
#squad_training_ds_path: str = const.DATASETS_FOLDER + "train-v2.0.json"
squad_testing_ds_path: str = const.DATASETS_FOLDER + "dev-v2.0.json"

In [3]:
#squad_training_ds_json: dict = {}
squad_testing_ds_json: dict = {}

In [4]:
#with open(squad_training_ds_path, 'r') as f:
#    squad_training_ds_json = json.load(f)

with open(squad_testing_ds_path, 'r') as f:
    squad_testing_ds_json = json.load(f)

In [5]:
#squad_training_ds_json.keys()

In [6]:
def squad_to_csv(json_dict: dict):
    squad_df = None
    root_dict = json_dict['data']
    root_items_len = len(root_dict)
    root_items_processed = 0
    start_time = dt.now()
    end_time = None
    for sub_dict_1 in root_dict:
        progress_msg = "Converting Squad JSON to csv... (Topic: " + str(root_items_processed + 1) + "/" + str(root_items_len)
        if end_time is None:
            progress_msg += ", elapsed time: 00:00)"
        else:
            #print(end_time)
            elapsed_time = end_time - start_time
            avg_time = elapsed_time / root_items_processed
            time_remaining = avg_time * (root_items_len - root_items_processed)
            progress_msg += ", elasped time: " + str(elapsed_time) + ", time remaining: " + str(time_remaining) + ")"
        print(progress_msg, end="                                                                                                      \r")

        title = sub_dict_1['title']
        paragraphs_dict = sub_dict_1['paragraphs']
        for sub_dict_2 in paragraphs_dict:
            paragraph = sub_dict_2['context']
            qas_dict = sub_dict_2['qas']
            for item in qas_dict:
                temp_df = None
                question = item['question']
                question_id = item['id']
                is_impossible = item['is_impossible']
                answers = []
                expanded_answers = []
                if is_impossible == False:
                    answers_dict = item['answers']
                else:
                    answers_dict = item['plausible_answers']
                    
                for answer in answers_dict:
                    answers.append(answer['text'])
                    expanded_answer = paragraph[269:]
                    expanded_answers.append(expanded_answer)
                temp_df = pd.DataFrame(data={
                    'source': "Squad 2.0", 
                    'topic': title,
                    "paragraph": paragraph, 
                    "question": question, 
                    "question_id": question_id, 
                    'is_impossible': is_impossible,
                    "answers": answers,
                    "expanded_answers": expanded_answers
                })
                if squad_df is None:
                    squad_df = temp_df
                else:
                    squad_df = pd.concat([squad_df, temp_df], ignore_index=True)
        end_time = dt.now()
        root_items_processed += 1
    print("Done                                                                                                               ")
    return squad_df
                

In [7]:
#squad_training_df = squad_to_csv(squad_training_ds_json)
squad_testing_df = squad_to_csv(squad_testing_ds_json)

Done                                                                                                                                                                                                              


In [8]:
squad_testing_df.head()

,source,topic,paragraph,question,question_id,is_impossible,answers,expanded_answers
0,Squad 2.0,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,False,France,"and and Norway who, under their leader Rollo, ..."
1,Squad 2.0,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,False,France,"and and Norway who, under their leader Rollo, ..."
2,Squad 2.0,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,False,France,"and and Norway who, under their leader Rollo, ..."
3,Squad 2.0,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,56ddde6b9a695914005b9628,False,France,"and and Norway who, under their leader Rollo, ..."
4,Squad 2.0,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,56ddde6b9a695914005b9629,False,10th and 11th centuries,"and and Norway who, under their leader Rollo, ..."


In [9]:
#squad_training_df = squad_to_csv(squad_training_ds_json)
squad_testing_df.to_csv(const.DATASETS_FOLDER + "squad_keyword_test.csv")